In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Score meanings**
The scores are in the 0-1 range with increments of 0.25 with the following meanings:

- 1.0 - Very close match. This is typically an exact match except possibly for differences in conjugation, quantity (e.g. singular vs. plural), and addition or removal of stopwords (e.g. “the”, “and”, “or”).
- 0.75 - Close synonym, e.g. “mobile phone” vs. “cellphone”. This also includes abbreviations, e.g. "TCP" -> "transmission control protocol".
- 0.5 - Synonyms which don’t have the same meaning (same function, same properties). This includes broad-narrow (hyponym) and narrow-broad (hypernym) matches.
- 0.25 - Somewhat related, e.g. the two phrases are in the same high level domain but are not synonyms. This also includes antonyms.
- 0.0 - Unrelated.


**Files**

- train.csv - the training set, containing phrases, contexts, and their similarity scores
- test.csv - the test set set, identical in structure to the training set but without the score
- sample_submission.csv - a sample submission file in the correct format

**Columns**

- id - a unique identifier for a pair of phrases
- anchor - the first phrase
- target - the second phrase
- context - the CPC classification (version 2021.05), which indicates the subject within which the similarity is to be scored
- score - the similarity. This is sourced from a combination of one or more manual expert ratings.


In [ ]:
train = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
test = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")
submission = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv")

In [ ]:
train.shape, test.shape, submission.shape

In [ ]:
train.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train['anchor'].value_counts()

In [ ]:
train[train['score'] == 1].head()

In [ ]:
train[train['score'] == 0.75].head()

In [ ]:
train[train['score'] == 0.5].head()

In [ ]:
train[train['score'] == 0.25].head()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.countplot(train['score']);

In [ ]:
x = train['target']
y = train['score']

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
token = Tokenizer()
token.fit_on_texts(x)

token_seq = token.texts_to_sequences(x)
token_pad = pad_sequences(token_seq, maxlen=300)

In [ ]:
vocab_size = len(token.word_index)+1
vocab_size

In [ ]:
glove_path = "../input/glove840b300dtxt/glove.840B.300d.txt"

In [ ]:
from tqdm import tqdm

embedding_vector = {}
f = open(glove_path)
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

In [ ]:
embedding_matrix = np.zeros((vocab_size,300))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

In [ ]:
from tensorflow.keras.layers import Dense, Bidirectional, Embedding, LSTM
from tensorflow.keras import Sequential

In [ ]:
# buliding the model
model = Sequential()
model.add(Embedding(vocab_size,300,weights = [embedding_matrix],input_length=300,trainable = False))
model.add(Bidirectional(LSTM(75)))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

In [ ]:
history = model.fit(token_pad,y,epochs = 50,batch_size=32)

In [ ]:
epochs = range(50)
plt.plot(history.history['loss'], epochs)

In [ ]:
# model saving

from keras.models import load_model
model.save("baseline_lstm.h5")   

In [ ]:
# load model    
model = load_model('./baseline_lstm.h5')

In [ ]:
# predict on testing data
test.head()

In [ ]:
x_test = test['target']
x_test = token.texts_to_sequences(x_test)

test_seq = pad_sequences(x_test,maxlen=300)

In [ ]:
predictions = model.predict(test_seq)

In [ ]:
test['score'] = predictions

In [ ]:
test.head()

In [ ]:
submission_data = test[['id','score']]
submission_data.head()

In [ ]:
submission_data.to_csv("submission.csv", index=False)